In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# ------------------------------
# Parameters
# ------------------------------
ETFS    = ["XLY","XLP","XLE","XLF","XLV","XLI","XLB","XLK","XLU"]
START   = "2015-01-01"
LOOKBACK_MONTHS = 6
TOP_N   = 3
TC      = 0.001  # transaction cost per round-trip

In [ ]:
# ------------------------------
# Download and prepare data
# ------------------------------
prices = yf.download(ETFS + ["SPY"], start=START, auto_adjust=True)["Close"]
returns = prices.pct_change().fillna(0)

In [ ]:
price_monthly = prices[ETFS].resample("ME").last()
momentum = price_monthly / price_monthly.shift(LOOKBACK_MONTHS) - 1
rank_pct = momentum.rank(pct=True, axis=1)

In [ ]:
signal = (rank_pct >= (1 - TOP_N / len(ETFS))).astype(float)
positions = signal.reindex(prices.index, method="ffill")
positions = positions.div(positions.sum(axis=1), axis=0).fillna(0)

In [ ]:
# ------------------------------
# Strategy returns
# ------------------------------
strategy_returns = (positions.shift() * returns[ETFS]).sum(axis=1)
strategy_returns -= positions.diff().abs().sum(axis=1) * TC
cumulative_strategy = (1 + strategy_returns).cumprod()

In [ ]:
# Benchmark: SPY buy-hold
spy_returns = returns["SPY"]
cumulative_spy = (1 + spy_returns).cumprod().reindex(cumulative_strategy.index, method="ffill")

In [ ]:
# ------------------------------
# Performance metrics function
# ------------------------------
def perf_stats(returns_series, cumulative_series):
    ann = returns_series.mean() * 252
    vol = returns_series.std(ddof=0) * np.sqrt(252)
    sharpe = ann / vol if vol != 0 else np.nan
    drawdown = (cumulative_series.cummax() - cumulative_series).max()
    return ann, vol, sharpe, drawdown

In [ ]:
s_ann, s_vol, s_sharpe, s_dd = perf_stats(strategy_returns, cumulative_strategy)
b_ann, b_vol, b_sharpe, b_dd = perf_stats(spy_returns.loc[cumulative_strategy.index], cumulative_spy)

In [ ]:
print(f"{'Metric':20}{'Strategy':>12}{'SPY':>12}")
print("-" * 44)
print(f"{'Ann. Return':20}{s_ann:12.2%}{b_ann:12.2%}")
print(f"{'Ann. Volatility':20}{s_vol:12.2%}{b_vol:12.2%}")
print(f"{'Sharpe Ratio':20}{s_sharpe:12.2f}{b_sharpe:12.2f}")
print(f"{'Max Drawdown':20}{s_dd:12.2%}{b_dd:12.2%}")

In [ ]:
# ------------------------------
# Performance attribution
# ------------------------------
daily_contrib = positions.shift() * returns[ETFS]
total_contrib = daily_contrib.sum()
avg_weights = positions.mean()

In [ ]:
attr = pd.DataFrame({
    "Total Return": total_contrib,
    "Avg Weight": avg_weights
}).sort_values("Total Return", ascending=False)

In [ ]:
print("\n=== Sector Attribution ===")
print(attr.to_string(float_format="{:.2%}".format))

In [ ]:
# Plot attribution
fig, ax1 = plt.subplots(figsize=(8,4))
attr["Total Return"].plot.bar(ax=ax1, rot=45, position=0, width=0.4, label="Total Return")
ax2 = ax1.twinx()
attr["Avg Weight"].plot.bar(ax=ax2, rot=45, position=1, width=0.4, label="Avg Weight")
ax1.set_ylabel("Total Return")
ax2.set_ylabel("Avg Weight")
ax1.set_title("Sector Attribution")
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
plt.tight_layout()
plt.show()

In [ ]:
# ------------------------------
# Parameter sweep and heatmap
# ------------------------------
lookbacks = [3, 6, 9, 12]
top_ns = [2, 3, 4, 5]
results = []

In [ ]:
for lb in lookbacks:
    mom = price_monthly / price_monthly.shift(lb) - 1
    pct = mom.rank(pct=True, axis=1)
    for tn in top_ns:
        sig = (pct >= (1 - tn / len(ETFS))).astype(float)
        pos = sig.reindex(prices.index, method="ffill")
        pos = pos.div(pos.sum(axis=1), axis=0).fillna(0)
        ret = (pos.shift() * returns[ETFS]).sum(axis=1)
        ret -= pos.diff().abs().sum(axis=1) * TC
        cum = (1 + ret).cumprod()
        ann = ret.mean() * 252
        vol = ret.std(ddof=0) * np.sqrt(252)
        sharpe = ann / vol if vol != 0 else np.nan
        maxdd = (cum.cummax() - cum).max()
        results.append({"lookback": lb, "top_n": tn,
                        "Sharpe": sharpe})

In [ ]:
df_res = pd.DataFrame(results)
df_pivot = df_res.pivot(index="lookback", columns="top_n", values="Sharpe")

In [ ]:
# Plot heatmap with labels and annotations
fig, ax = plt.subplots(figsize=(6, 4))
data = df_pivot.values
im = ax.imshow(data, origin="lower", aspect="auto", cmap="viridis")

In [ ]:
ax.set_xticks(np.arange(data.shape[1]))
ax.set_yticks(np.arange(data.shape[0]))
ax.set_xticklabels(df_pivot.columns)
ax.set_yticklabels(df_pivot.index)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

In [ ]:
cbar = fig.colorbar(im, ax=ax, label="Sharpe Ratio")

In [ ]:
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        ax.text(j, i, f"{data[i, j]:.2f}", ha="center", va="center",
                color="white" if data[i, j] < data.max()/2 else "black")

In [ ]:
ax.set_xlabel("Top N ETFs")
ax.set_ylabel("Lookback (months)")
ax.set_title("Sharpe Ratio Heatmap")
plt.tight_layout()
plt.show()